In [322]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import math

In [323]:
telecom_df = pd.read_csv('telecom_churn_data.csv')

In [324]:
telecom_df.info()
#Most are numeric, some of object type. Check these


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Columns: 226 entries, mobile_number to sep_vbc_3g
dtypes: float64(179), int64(35), object(12)
memory usage: 172.4+ MB


In [325]:
#print(telecom_df.select_dtypes(include=['object']).head())
#These are all of type data, of format mm/dd/yyyy, conver to dates
#for all date columns, keep only the day of the month, since month is present in the
#column name, year is common for all.
date_columns = list(telecom_df.select_dtypes(include=['object']).columns)
for column in date_columns:
    telecom_df[column] = pd.to_datetime(telecom_df[column], format='%m/%d/%Y')
    telecom_df[column] = telecom_df[column].dt.day

In [326]:
#Those who have recharged with an amount more than or equal to X, 
#where X is the 70th percentile of the average recharge amount in the first two months (the good phase).
'''
print(telecom_df['total_rech_amt_6'].isnull().sum()) 0;
print(telecom_df['total_rech_amt_7'].isnull().sum()) 0;
print(telecom_df['total_rech_num_6'].isnull().sum()) 0;
print(telecom_df['total_rech_num_7'].isnull().sum()) 0;
'''
#since, all values for the columns used below are non-nulls, we need not do any imputation here
if 'av_rech_67' not in list(telecom_df.columns):
    telecom_df['av_rech_67'] =  (telecom_df['total_rech_amt_6']+telecom_df['total_rech_amt_7'])/(telecom_df['total_rech_num_6']+telecom_df['total_rech_num_7'])
    avg_70_quantile = telecom_df.av_rech_67.quantile(q=0.7)
    telecom_df = telecom_df[telecom_df.av_rech_67>= avg_70_quantile]
print ('Data is now at 70th quantile of average recharge for the first 2 months. Total rows remaining: '+ str(len(telecom_df)))


Data is now at 70th quantile of average recharge for the first 2 months. Total rows remaining: 29944


In [327]:
telecom_df.isnull().sum().describe()
# telecom_df.isnull().sum()>0).sum()
# 166 columns with missing values
# A lot of values are missing in many colums. Cannot remove such rows. Will have to find a way to impute

count      227.000000
mean      4205.832599
std       7207.678037
min          0.000000
25%          0.000000
50%        989.000000
75%       2083.000000
max      19826.000000
dtype: float64

In [328]:
#Find columns with only single non-null value
redundant_cols = [col for col in telecom_df.columns if telecom_df[col].nunique()==1]
telecom_df = telecom_df.drop(redundant_cols, axis=1)
print('Following columns were removed: \n',redundant_cols)


Following columns were removed: 
 ['circle_id', 'loc_og_t2o_mou', 'std_og_t2o_mou', 'loc_ic_t2o_mou', 'last_date_of_month_6', 'last_date_of_month_7', 'last_date_of_month_8', 'last_date_of_month_9', 'std_og_t2c_mou_6', 'std_og_t2c_mou_7', 'std_og_t2c_mou_8', 'std_og_t2c_mou_9', 'std_ic_t2o_mou_6', 'std_ic_t2o_mou_7', 'std_ic_t2o_mou_8', 'std_ic_t2o_mou_9']


In [329]:
#Find rows with null dates
for month in [6,7,8,9]:
    last_rech_col = 'date_of_last_rech_'+str(month)
    num_recharges_col = 'total_rech_num_'+str(month)
    nullrechdate_df = telecom_df[telecom_df['date_of_last_rech_9'].isnull()]
    if (len(nullrechdate_df[~nullrechdate_df.total_rech_num_9==0])>0):
        print('Nulls found for date in month:', month)
    else:
        print('All nulls in month ',month, ' correspond to no recharges done.')
    
#Hence, nans in date_of_last_rech_ columns are all valid. Giving value > 32 so that we know it is invalid


#dates can be changed to categorical values

#NaN can be assumed to be Recharge not done 

# def process(x):
#     if type(x) is float:
#         return 0
#     else: 
#         return 1

# for col in dc:
#     dat[col] = dat[col].apply(process)

All nulls in month  6  correspond to no recharges done.
All nulls in month  7  correspond to no recharges done.
All nulls in month  8  correspond to no recharges done.
All nulls in month  9  correspond to no recharges done.


In [330]:
for month in [6,7,8,9]:
    max_rech_data = 'max_rech_data_'+str(month)
    av_rech_amt_data='av_rech_amt_data_'+str(month)
    total_rech_data = 'total_rech_data_'+str(month)
    date_of_last_rech_data = 'date_of_last_rech_data_'+str(month)
    count_rech_3g = 'count_rech_3g_'+str(month)
    count_rech_2g = 'count_rech_2g_'+str(month)
    arpu_3g = 'arpu_3g_'+str(month)
    arpu_2g = 'arpu_2g_'+str(month)
    fb_user = 'fb_user_'+str(month)
    night_pck_user = 'night_pck_user_' +str(month)

    #Verify, that if toatl rech_data is null, then max, av, date, count_3g, count_2g are null too 
    #nonnullrows = np.sum(np.sum(telecom_df[telecom_df[total_rech_data].isnull()]
    #if last date for data recharge is null, then verify if other elements corresponding to null are null/0 too. If yes,
    # we can assign all of them as 0.
    
    max_rech_nonzerorows = len(telecom_df[(telecom_df[date_of_last_rech_data].isnull()) & ((~telecom_df[max_rech_data].isnull()) | (telecom_df[max_rech_data]>0))])
    if max_rech_nonzerorows == 0:
        telecom_df.loc[telecom_df[date_of_last_rech_data].isnull(),max_rech_data] = 0

    #Similarily for rest columns too
    assumption=True
    for data_colm in [total_rech_data, av_rech_amt_data,count_rech_3g,count_rech_2g,arpu_3g,arpu_2g,fb_user,night_pck_user]:
        rech_colm_nonzerorows = len(telecom_df[(telecom_df[date_of_last_rech_data].isnull()) & ((~telecom_df[data_colm].isnull()) | (telecom_df[data_colm]>0))])
        if rech_colm_nonzerorows == 0:
            telecom_df.loc[telecom_df[date_of_last_rech_data].isnull(),data_colm] = 0
        else:
            assumption = False
            print('Wrong assumption for', data_colm)
    if assumption == True:
#       If assumption was correct, then provide invalid date, or -ve
        telecom_df.loc[telecom_df[date_of_last_rech_data].isnull(),date_of_last_rech_data] = -1


In [331]:
def makeZeroIfTotalIsZero(colm, total_colm, typeofcol):
    total_ = np.sum(telecom_df[telecom_df[colm].isnull()][total_colm])
    if total_ == 0:
        #('Total for ', typeofcol, ' is 0. Hence make all null values 0')
        telecom_df.loc[telecom_df[colm].isnull(),colm] = 0

for month in [6,7,8,9]:
    total_ic_mou = 'total_ic_mou_'+str(month)
    roam_ic_mou = 'roam_ic_mou_'+str(month)
    loc_ic_t2t_mou = 'loc_ic_t2t_mou_'+str(month)
    loc_ic_t2m_mou = 'loc_ic_t2m_mou_'+str(month)
    loc_ic_t2f_mou = 'loc_ic_t2f_mou_'+str(month)
    loc_ic_mou  = 'loc_ic_mou_'+str(month)
    std_ic_t2t_mou = 'std_ic_t2t_mou_'+str(month)
    std_ic_t2m_mou = 'std_ic_t2m_mou_'+str(month)
    std_ic_t2f_mou = 'std_ic_t2f_mou_'+str(month)
    std_ic_mou  =  'std_ic_mou_'+str(month)
    spl_ic_mou = 'spl_ic_mou_'+str(month)
    isd_ic_mou = 'isd_ic_mou_'+str(month)
    ic_others  = 'ic_others_'+str(month)

    total_og_mou = 'total_og_mou_' + str(month)
    roam_og_mou = 'roam_og_mou_' + str(month)
    loc_og_t2t_mou = 'loc_og_t2t_mou_' + str(month)
    loc_og_t2m_mou = 'loc_og_t2m_mou_' + str(month)
    loc_og_t2f_mou = 'loc_og_t2f_mou_' + str(month)
    loc_og_t2c_mou = 'loc_og_t2c_mou_' + str(month)
    loc_og_mou = 'loc_og_mou_' + str(month)
    std_og_t2t_mou = 'std_og_t2t_mou_' + str(month)
    std_og_t2m_mou = 'std_og_t2m_mou_' + str(month)
    std_og_t2f_mou = 'std_og_t2f_mou_' + str(month)
    std_og_mou = 'std_og_mou_' + str(month)
    isd_og_mou = 'isd_og_mou_' + str(month)
    spl_og_mou = 'spl_og_mou_' + str(month)
    og_others = 'og_others_' + str(month)
      
    onnet_mou = 'onnet_mou_'+str(month)
    offnet_mou = 'offnet_mou_'+str(month)

    date_of_last_rech = 'date_of_last_rech_' + str(month)
    total_rech_amt = 'total_rech_amt_'+ str(month)
    #Incoming columns
    for colm in [roam_ic_mou, loc_ic_t2t_mou, loc_ic_t2m_mou, 
                 loc_ic_t2f_mou, loc_ic_mou, std_ic_t2t_mou, 
                 std_ic_t2m_mou, std_ic_t2f_mou, std_ic_mou,
                 spl_ic_mou, isd_ic_mou, ic_others]:
        makeZeroIfTotalIsZero(colm, total_ic_mou, 'incoming')
    
    #Outgoing columns

    for colm in [roam_og_mou,loc_og_t2t_mou,loc_og_t2m_mou,
                 loc_og_t2f_mou,loc_og_t2c_mou,loc_og_mou,
                 std_og_t2t_mou,std_og_t2m_mou,std_og_t2f_mou,
                 std_og_mou,isd_og_mou,spl_og_mou,og_others]:
        makeZeroIfTotalIsZero(colm, total_ic_mou, 'outgoing')
    
    #Onnet columns
    #If outgoing + incoming mou for the nuxll entries is 0, then onnet, offnet mou are also 0
    outgoing_mou_when_onnet_null = np.sum(telecom_df[telecom_df[onnet_mou].isnull()][total_og_mou])
    incoming_mou_when_onnet_null = np.sum(telecom_df[telecom_df[onnet_mou].isnull()][total_ic_mou])
    onnet = outgoing_mou_when_onnet_null+incoming_mou_when_onnet_null
    #'Total onnet(calculated) for month: ', str(month), ' is: ', onnet, ' Actual null'..... #0
    telecom_df.loc[telecom_df[onnet_mou].isnull(),onnet_mou] = 0
    
    outgoing_mou_when_offnet_null = np.sum(telecom_df[telecom_df[offnet_mou].isnull()][total_og_mou])
    incoming_mou_when_offnet_null = np.sum(telecom_df[telecom_df[offnet_mou].isnull()][total_ic_mou])
    offnet = outgoing_mou_when_offnet_null+incoming_mou_when_offnet_null
    #'Total offnet(calculated) for month: ', str(month), ' is: ', offnet, ' Actual null'.... #0
    telecom_df.loc[telecom_df[offnet_mou].isnull(),offnet_mou] = 0
    
    
    total_ = np.sum(telecom_df[telecom_df[date_of_last_rech].isnull()][total_rech_amt])
    if total_ == 0:
        telecom_df.loc[telecom_df[date_of_last_rech].isnull(),date_of_last_rech] = -1

In [332]:
(telecom_df.isnull().sum()>0).sum() # 0 
#Every column has been handled. No nulls remaining

0

In [333]:
#Identify churners now
# total_ic_mou_9
# total_og_mou_9
# vol_2g_mb_9
# vol_3g_mb_9
telecom_df['churn'] = np.zeros(len(telecom_df))
telecom_df.loc[((telecom_df['total_ic_mou_9'] == 0) & 
                           (telecom_df['total_og_mou_9'] == 0) &
                           (telecom_df['vol_2g_mb_9'] == 0) &
                           (telecom_df['vol_3g_mb_9'] == 0) 
               ),'churn'] = 1
#Remove everything with _9
colms_with_9 = [colm for colm in telecom_df.columns if colm.endswith('_9')]
telecom_df = telecom_df.drop(colms_with_9, axis=1)

In [334]:
#Build meta features
#Identify churners in the 3rd month

#RFM
# so, features are to be built from month 6 and 7 itself.
telecom_df['recency_goodphase'] = np.ones(len(telecom_df))*61
telecom_df['recency_goodphase_data'] = np.ones(len(telecom_df))*61


In [335]:
#Regular recharge
telecom_df.loc[telecom_df.date_of_last_rech_7>0,'recency_goodphase']= 31 - telecom_df[telecom_df.date_of_last_rech_7 > 0].date_of_last_rech_7
telecom_df.loc[(telecom_df.date_of_last_rech_7<0)&(telecom_df.date_of_last_rech_6>0),'recency_goodphase']= 31 + (30 - telecom_df[(telecom_df.date_of_last_rech_7<0)&(telecom_df.date_of_last_rech_6 > 0)].date_of_last_rech_6)
                                                                            
                 
#Data recharge
telecom_df.loc[telecom_df.date_of_last_rech_data_7>0,'recency_goodphase_data']= 31 - telecom_df[telecom_df.date_of_last_rech_data_7 > 0].date_of_last_rech_data_7
telecom_df.loc[(telecom_df.date_of_last_rech_data_7<0)&(telecom_df.date_of_last_rech_data_6>0),'recency_goodphase_data']= 31 + (30 - telecom_df[(telecom_df.date_of_last_rech_data_7<0)&(telecom_df.date_of_last_rech_data_6 > 0)].date_of_last_rech_data_6)

                                                                            
                                                                            

In [337]:
telecom_df[telecom_df.recency_goodphase>=60][['date_of_last_rech_7','date_of_last_rech_6','date_of_last_rech_data_7',
  'date_of_last_rech_data_6','recency_goodphase','recency_goodphase_data']]




,date_of_last_rech_7,date_of_last_rech_6,date_of_last_rech_data_7,date_of_last_rech_data_6,recency_goodphase,recency_goodphase_data
3274,-1.0,1.0,-1.0,-1.0,60.0,61.0
5932,-1.0,1.0,-1.0,-1.0,60.0,61.0
10331,-1.0,1.0,-1.0,-1.0,60.0,61.0
10575,-1.0,1.0,-1.0,1.0,60.0,60.0
50080,-1.0,1.0,-1.0,-1.0,60.0,61.0
53228,-1.0,1.0,-1.0,-1.0,60.0,61.0
54133,-1.0,1.0,-1.0,-1.0,60.0,61.0
77195,-1.0,1.0,-1.0,-1.0,60.0,61.0


In [ ]:
#Total Revenue
telecom_df['revenue_lost_goodphase'] = np.zeros(len(telecom_df))


In [161]:
reg_colms = [colm for colm in telecom_df.columns if not (colm.endswith('_6') or colm.endswith('_7') or colm.endswith('_8'))]
# 'aon', 'aug_vbc_3g', 'jul_vbc_3g', 'jun_vbc_3g', 'sep_vbc_3g'
print(reg_colms)
colms_with_6 = [colm for colm in telecom_df.columns if (colm.endswith('_6'))]
#Based on the third month how recently has the customer made any recharge
print(colms_with_6)

['mobile_number', 'aon', 'aug_vbc_3g', 'jul_vbc_3g', 'jun_vbc_3g', 'sep_vbc_3g', 'av_rech_67', 'churn']
['arpu_6', 'onnet_mou_6', 'offnet_mou_6', 'roam_ic_mou_6', 'roam_og_mou_6', 'loc_og_t2t_mou_6', 'loc_og_t2m_mou_6', 'loc_og_t2f_mou_6', 'loc_og_t2c_mou_6', 'loc_og_mou_6', 'std_og_t2t_mou_6', 'std_og_t2m_mou_6', 'std_og_t2f_mou_6', 'std_og_mou_6', 'isd_og_mou_6', 'spl_og_mou_6', 'og_others_6', 'total_og_mou_6', 'loc_ic_t2t_mou_6', 'loc_ic_t2m_mou_6', 'loc_ic_t2f_mou_6', 'loc_ic_mou_6', 'std_ic_t2t_mou_6', 'std_ic_t2m_mou_6', 'std_ic_t2f_mou_6', 'std_ic_mou_6', 'total_ic_mou_6', 'spl_ic_mou_6', 'isd_ic_mou_6', 'ic_others_6', 'total_rech_num_6', 'total_rech_amt_6', 'max_rech_amt_6', 'date_of_last_rech_6', 'last_day_rch_amt_6', 'date_of_last_rech_data_6', 'total_rech_data_6', 'max_rech_data_6', 'count_rech_2g_6', 'count_rech_3g_6', 'av_rech_amt_data_6', 'vol_2g_mb_6', 'vol_3g_mb_6', 'arpu_3g_6', 'arpu_2g_6', 'night_pck_user_6', 'monthly_2g_6', 'sachet_2g_6', 'monthly_3g_6', 'sachet_3g_6

In [208]:
print(total)

,total_rech_amt_6,max_rech_amt_6,last_day_rch_amt_6
0,362,252,252
7,1580,1580,0
9,220,110,110
13,507,110,110
16,570,110,100
19,154,154,154
20,437,250,120
22,255,110,37
23,0,0,0
24,440,110,110


In [338]:
colms_with_data_6 = [colm for colm in telecom_df.columns if (colm.startswith('arpu'))]
colms_with_data_6

['arpu_6',
 'arpu_7',
 'arpu_8',
 'arpu_3g_6',
 'arpu_3g_7',
 'arpu_3g_8',
 'arpu_2g_6',
 'arpu_2g_7',
 'arpu_2g_8']

In [343]:
telecom_df[['av_rech_amt_data_6','count_rech_data_6']]

KeyError: "['count_rech_data_6'] not in index"

In [195]:
len(telecom_df[(telecom_df.date_of_last_rech_data_7.isnull()) & ((~telecom_df.max_rech_data_7.isnull()) | (telecom_df.max_rech_data_7>0))])

0

In [ ]:
telecom_df